In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/deberta-v3-ai-generated/checkpoint-10000-base-50w-93-0112/config.json
/kaggle/input/deberta-v3-ai-generated/checkpoint-10000-base-50w-93-0112/trainer_state.json
/kaggle/input/deberta-v3-ai-generated/checkpoint-10000-base-50w-93-0112/training_args.bin
/kaggle/input/deberta-v3-ai-generated/checkpoint-10000-base-50w-93-0112/scheduler.pt
/kaggle/input/deberta-v3-ai-generated/checkpoint-10000-base-50w-93-0112/model.safetensors
/kaggle/input/deberta-v3-ai-generated/checkpoint-10000-base-50w-93-0112/rng_state.pth
/kaggle/input/deberta-v3-ai-generated/debertav3-base-tokenizer/spm.model
/kaggle/input/deberta-v3-ai-generated/debertav3-base-tokenizer/config.json
/kaggle/input/deberta-v3-ai-generated/debertav3-base-tokenizer/tokenizer_config.json
/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.

In [2]:
from torch.nn.functional import softmax
import torch
from transformers import DebertaV2Tokenizer, DebertaV2ForSequenceClassification

In [3]:
tokenizer_path = "/kaggle/input/deberta-v3-ai-generated/debertav3-base-tokenizer"
model_name = "/kaggle/input/deberta-v3-ai-generated/checkpoint-10000-base-50w-93-0112"
tokenizer = DebertaV2Tokenizer.from_pretrained(tokenizer_path)
model = DebertaV2ForSequenceClassification.from_pretrained(model_name)

In [4]:
model

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine

In [5]:
df = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/test_essays.csv")
df.head()

,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [6]:
submission = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv")
submission.head()

,id,generated
0,0000aaaa,0.1
1,1111bbbb,0.9
2,2222cccc,0.4


In [7]:
def predict_ai_generated(text):
    inputs = tokenizer(
        text=text, 
        return_tensors="pt", 
        truncation=True, 
        max_length=512
    )
    with torch.no_grad():
        outputs = model(**inputs)
    probs = softmax(outputs.logits, dim=1)
    return probs[0, 1].item()

df['generated'] = df['text'].apply(predict_ai_generated)
df.head()

,id,prompt_id,text,generated
0,0000aaaa,2,Aaa bbb ccc.,0.165391
1,1111bbbb,3,Bbb ccc ddd.,0.158366
2,2222cccc,4,CCC ddd eee.,0.136918


In [8]:
submission['generated'] = df['generated']
submission.head()

,id,generated
0,0000aaaa,0.165391
1,1111bbbb,0.158366
2,2222cccc,0.136918


In [9]:
submission.to_csv("submission.csv", index=False)